In [1]:
"""
IMPORTS
"""
import pandas as pd
import xlrd
import seaborn as sns
import matplotlib.pyplot as plt



In [2]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline 

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

In [9]:
df_2020_to_resample= pd.read_csv("E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Outcomes\df_2020_to_resample.csv",sep=',')

In [10]:
df_2020_to_resample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24351 entries, 0 to 24350
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Código Tipo Supuesto Urgente  24351 non-null  float64
 1   Código Distrito Centro        24351 non-null  int64  
 2   Age_range_code                24351 non-null  int64  
 3   Nationality_code              24351 non-null  int64  
 4   Sexo_code                     24351 non-null  int64  
 5   year                          24351 non-null  int64  
 6   Quarteryear                   24351 non-null  int64  
dtypes: float64(1), int64(6)
memory usage: 1.3 MB


### 1.- RESAMPLING DATASET

Vamos a eliminar todas las columnas que no sean validas.

In [11]:
df_2020_to_resample.head()

,Código Tipo Supuesto Urgente,Código Distrito Centro,Age_range_code,Nationality_code,Sexo_code,year,Quarteryear
0,50.0,2,3,0,0,2020,1
1,40.0,2,6,0,1,2020,1
2,30.0,2,4,1,0,2020,1
3,40.0,2,6,0,0,2020,1
4,50.0,21,2,0,1,2020,1


In [12]:
strategy_u = {50.0: 743, 40.0:2236, 30.0:2016, 20.0:23, 10.0:290}
#strategy_o = {50.0: 4000, 40.0:3000, 30.0:2500, 20.0:2344, 10.0:3000}

In [14]:
X=df_2020_to_resample[['Código Distrito Centro','Age_range_code','Nationality_code','Sexo_code']]
y=df_2020_to_resample['Código Tipo Supuesto Urgente']

In [15]:
under=RandomUnderSampler(sampling_strategy=strategy_u)
#over=SMOTE(sampling_strategy=strategy_o)
steps=[('u',under)]#,('o',over)]
pipeline= Pipeline(steps=steps)
X,y=pipeline.fit_resample(X,y)

In [17]:
df_2020=pd.DataFrame()
df_2020['Código Distrito Centro']=X['Código Distrito Centro']
df_2020['Age_range_code']=X['Age_range_code']
df_2020['Nationality_code']=X['Nationality_code']
df_2020['Sexo_code']=X['Sexo_code']
df_2020['Código Tipo Supuesto Urgente']=y
df_2020



,Código Distrito Centro,Age_range_code,Nationality_code,Sexo_code,Código Tipo Supuesto Urgente
0,6,2,1,0,10.0
1,2,2,0,0,10.0
2,13,0,1,1,10.0
3,8,0,1,1,10.0
4,13,0,1,1,10.0
...,...,...,...,...,...
5303,6,1,1,0,50.0
5304,6,0,1,1,50.0
5305,6,1,1,0,50.0
5306,6,2,1,1,50.0


In [18]:
df_2020['Código Tipo Supuesto Urgente'].value_counts()

40.0    2236
30.0    2016
50.0     743
10.0     290
20.0      23
Name: Código Tipo Supuesto Urgente, dtype: int64

In [19]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5308 entries, 0 to 5307
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Código Distrito Centro        5308 non-null   int64  
 1   Age_range_code                5308 non-null   int64  
 2   Nationality_code              5308 non-null   int64  
 3   Sexo_code                     5308 non-null   int64  
 4   Código Tipo Supuesto Urgente  5308 non-null   float64
dtypes: float64(1), int64(4)
memory usage: 207.5 KB


In [21]:
df_2020['year']=2020

In [22]:
df_2020.to_csv('E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Outcomes\df_2020_balanced.csv',header=True,index=False)

### 2.- RECOLLECTING DATAS

In [ ]:
df_2020_balanced= pd.read_csv("E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Outcomes\df_2020_balanced.csv",sep=',')
df_urgencias_to_resample = pd.read_csv("E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Outcomes\df_urgencias_to_resample.csv",sep=',')
df_urgencias_resampled=pd.concat([df_urgencias_to_resample,df_2020_balanced], ignore_index=True)
df_urgencias_resampled.head()

### 3.- Exploring Graphics

Vamos a hacer unas gráficas para valorar los datos que disponemos.

Más gráficas en __Tableau__

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df_urgencias_resampled.corr(), annot=True);

Se observa que hay cierta correlación entre los parámetros de **year**, **Nationality_code**,**Código Distrito Centro**.

También se observa que no hay aparente correlación con los parámetros **Código Tipo Atención**,**Age_range_code** y **Sex_code**.

**year** también parece tener correlación con otros parámetros como **Nationality_code** o **Código Distrito centro**.

**Nationality code** y **Age_range_code** parece que también tiene una cierta correlación.


In [ ]:
sns.pairplot(df_urgencias_resampled,hue='Código Tipo Supuesto Urgente');

En este **pairplot** se ve claramente que los valores no son continuos, sino discretos.

Hay que tener esto en cuenta a la hora de realizar cualquier análisis y predicción, ya que habrá que hacer __clasificación__.

In [ ]:
plt.figure(figsize=(15,10))
sns.displot(df_urgencias_resampled['Código Tipo Supuesto Urgente'])
plt.xticks(np.arange(10,50,step=10));

In [ ]:
df_urgencias_resampled.to_csv('E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Outcomes\df_urgencias_totales_balanced.csv',header=True,index=False)

In [ ]:
df_modelado= df_urgencias_resampled

### 4.- Preparing TRAIN AND TEST DataFrames

In [ ]:
X=df_modelado[['Código Distrito Centro','Age_range_code','Nationality_code','Sexo_code']]
y=df_modelado['Código Tipo Supuesto Urgente']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)       # Genero los datos para entrenamiento y testeo
print(X_train.shape)

print(X_test.shape)

print(y_train.shape)

print(y_test.shape)

In [ ]:
train=pd.concat([X_train,y_train],axis=1)
test=pd.concat([X_test,y_test],axis=1)

In [ ]:
train.to_csv('E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\df_train_balanced.csv',header=True,index=False)
test.to_csv('E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\df_test_balanced.csv',header=True,index=False)

### 5.- My first Model (DecisionTreeClassifier)

In [ ]:
train=pd.read_csv('E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\df_train_balanced.csv',sep=',')
test=pd.read_csv('E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\df_test_balanced.csv',sep=',')

X_test=test[['Código Distrito Centro','Age_range_code','Nationality_code','Sexo_code']]
X_train=train[['Código Distrito Centro','Age_range_code','Nationality_code','Sexo_code']]
y_test=test['Código Tipo Supuesto Urgente']
y_train=train['Código Tipo Supuesto Urgente']

In [ ]:
train['Código Tipo Supuesto Urgente'].value_counts()

In [ ]:
tree_reg = DecisionTreeClassifier(random_state=20)
tree_reg.fit(X_train, y_train)
y_pred = tree_reg.predict(X_test)


In [ ]:
def error_modelo(model):
    y_pred = model.predict(X_test)
    acc_model=accuracy_score(y_test, y_pred)
    precision_model=precision_score(y_test, y_pred,average='macro')
    recall_model=recall_score(y_test, y_pred,average='macro')
    roc_auc_model=roc_auc_score(y_test, model.predict_proba(X_test),multi_class='ovr')
    f1_model=f1_score(y_test, y_pred,average='macro')
    conf_model=confusion_matrix(y_test, y_pred, normalize='true')
    print('Accuracy', acc_model)
    print('Precision', precision_model)
    print('Recall', recall_model)
    print('ROC', roc_auc_model)
    print('F1', f1_model)

    plt.figure(figsize=(10,10))
    sns.heatmap(conf_model, annot=True)

In [ ]:
error_modelo(tree_reg)